In [77]:
import numpy as np
import subprocess as sp
import parmed as pmd
from openeye.oechem import *
from openeye.oeomega import *

from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

In [30]:
smiles = ['C1CCCCC1', 'CCO']
names = ['ZBE', 'ZXQ']
molecules = []

In [31]:
def process_smiles(string, name=None, add_hydrogens=True, add_tripos=True, charge=True):
    mol = OEMol()
    OESmilesToMol(mol, string)
    if add_hydrogens:
        OEAddExplicitHydrogens(mol)
    if add_tripos:
        OETriposAtomNames(mol)
    if charge:
        for atom in mol.GetAtoms():
            atom.SetPartialCharge(atom.GetFormalCharge())
    if name:
        mol.SetTitle(name)
    return mol

for (smile, name) in zip(smiles, names):
    molecules.append(process_smiles(smile, name))

Add some partial charges.

In [60]:
# Modified https://github.com/openforcefield/smirnoff/blob/40a630816968b99e54f0bb2e9fadee3e7e566601/smirnoff/forcefield.py
def assign_partial_charges(molecule, modifycharges=True):
    omega = OEOmega()
    omega.SetMaxConfs(800)
    omega.SetCanonOrder(False)
    omega.SetSampleHydrogens(True)
    omega.SetEnergyWindow(15.0)
    omega.SetRMSThreshold(1.0)
    omega.SetStrictStereo(True) #Don't generate random stereoisomer if not specified

    charged_copy = OEMol(molecule)
    status = omega(charged_copy)
    if not status:
        raise(RuntimeError("Omega returned error code %s" % status))

    # Assign charges
    status = openeye.oequacpac.OEAssignPartialCharges(charged_copy, getattr(oequacpac, 'OECharges_AM1BCCSym'), False, False)
    if not status:
        raise(RuntimeError("OEAssignPartialCharges returned error code %s" % status))
    # Our copy has the charges we want but not the right conformation. Copy charges over. Also copy over Wiberg bond orders if present
    partial_charges = []
    if modifycharges:
        for atom in charged_copy.GetAtoms():
            partial_charges.append( atom.GetPartialCharge() )
        for (idx,atom) in enumerate(molecule.GetAtoms()):
            atom.SetPartialCharge( partial_charges[idx] )
            
# Modified https://docs.eyesopen.com/toolkits/cookbook/python/modeling/am1-bcc.html
def print_partial_charges(molecule):
    absFCharge = 0
    sumFCharge = 0
    sumPCharge = 0.0
    for atom in molecule.GetAtoms():
        sumFCharge += atom.GetFormalCharge()
        absFCharge += abs(atom.GetFormalCharge())
        sumPCharge += atom.GetPartialCharge()
    print("{}: {} formal charges give total charge {}"
          "; sum of partial charges {:5.4f}".format(molecule.GetTitle(), absFCharge,
                                                    sumFCharge, sumPCharge))


In [61]:
for molecule in molecules:
    assign_partial_charges(molecule)
    print_partial_charges(molecule)

ZBE: 0 formal charges give total charge 0; sum of partial charges -0.0000
ZXQ: 0 formal charges give total charge 0; sum of partial charges -0.0000


Now, load in coordinates.

In [75]:
from simtk.openmm.app import PDBFile
# Loading with PDBFixer loses bond information for some reason...
fixer = PDBFile('original/cyclohexane_ethanol_0.4_0.6.pdb')

Load in the forcefield and create the system.

In [76]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml', 'forcefield/tip3p.ffxml') 
system = ff.createSystem(fixer.topology, molecules, 
                         nonbondedMethod=PME, 
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4,
                         rigidWater=False)

Try to save with ParmEd.

In [78]:
structure = pmd.openmm.topsystem.load_topology(fixer.topology, system, fixer.positions)

In [79]:
structure.save('generated/box_smirnoff.prmtop')

In [80]:
!ls -lh generated/box_smirnoff.prmtop

-rw-rw-r--. 1 dslochower dslochower 213M Feb  5 22:06 generated/box_smirnoff.prmtop
